In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@stretaildataproj.dfs.core.windows.net/orders")

In [0]:
df.display()
df.printSchema()

In [0]:
df=df.withColumn("order_date", col("order_date").cast("date"))\
    .withColumn("total_amount" , col("total_amount").cast("double"))\
    .withColumn("quantity" , col("quantity").cast("integer"))
df.printSchema()

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df = df.withColumn("order_date", to_timestamp(col("order_date")))
df.display()
df.printSchema()


In [0]:
df = df.withColumn("year" , year(col("order_date")))
df.display()


In [0]:
df.filter(col("order_date").isNull()).display()


In [0]:
class windows:
   
    def dense_rank(self ,df):
        df_dense_rank =  df.withColumn("flag" , dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

        return df_dense_rank
    
    def rank(self ,df):
        df_rank = df.withColumn("rank_flag" , rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

        return df_rank
    
    def row_number(self ,df)    :
        df_row_number = df.withColumn("row_flag" , row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

        return df_row_number

In [0]:
obj = windows()
obj.dense_rank(df_new).display()

In [0]:
df.printSchema()
df.write.format("delta").mode("append").save("abfss://silver@stretaildataproj.dfs.core.windows.net/orders/")   